In [1]:
import pandas as pd
import json, re
from tqdm import tqdm
from pkg import claude,config,embedding

# Vector Store

In [2]:
vs = embedding.load_vector_store("./spider/keyword_store")

/Users/ijh/Desktop/Code/sql-agent/pkg/embedding.py:121: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


# Example

In [3]:
with open('./spider/example.json') as f:
    examples_train = json.load(f)
    f.close()
    """
    {
        "original_question":{
            "origitnal_sql" = original_sql
        }
    }
    """

# Test Set

In [4]:
with open('./spider/test_spider.json') as f:
    dbs = json.load(f)
    f.close()
"""
    dbs[$db_id]
"""

'\n    dbs[$db_id]\n'

In [5]:
with open('./spider/spider_data/test.json') as f:
    test = json.load(f)
    f.close()
"""
    test[idx]['query']
    test[idx]['question']
    test[idx]['db_id']
"""

"\n    test[idx]['query']\n    test[idx]['question']\n    test[idx]['db_id']\n"

# Test Tables for Masking

In [6]:
test_tables = pd.read_json('./spider/spider_data/test_tables.json')

In [7]:
db_masking = {}
for db_idx in range(len(test_tables)):
    db = test_tables.loc[db_idx]
    db['db_id']
    # Masking Keyword : Table Name, Column Name
    masking_keyword = []
    
    for tab in db.table_names:
        masking_keyword.append(tab)
    for col in range(1,len(db.column_names)):
        masking_keyword.append(db.column_names[col][1])
    db_name = re.sub(r'\d+', '', db['db_id'])
    db_name = re.sub(r'[^a-zA-Z\'\-\s]', '', db_name)
    masking_keyword.append(db_name)
    db_masking[db.db_id] = masking_keyword

API KEY

In [8]:
api_key = config.get_API_keys('./test.key')

In [ ]:
res = []

# Main Code

In [73]:
mask_token = "[MASK]"
generator = claude.SQLQueryGenerator(api_key)
count = 5
for idx in tqdm(range(1865,len(test))):
    if count == 0: break
    t = test[idx]
    
    db_name = t['db_id']
    question = t['question']
    answer = t['query']
    schema = dbs[t['db_id']]['tables']
    foreign_keys = dbs[t['db_id']]['foreign_keys']    
    masked_question = t['question']
    examples = []
    
    for keyword in db_masking[db_name]:
        patterns = [rf'\b{re.escape(keyword)}\b',rf'\b{re.escape(keyword)}s?\b']
        for pattern in patterns:
            masked_question = re.sub(pattern, mask_token, masked_question) 
    
    similar_questions = embedding.search_keywords(vs, masked_question, k =3)
    for similar_question, _ in similar_questions:
        examples.append(
            {
            "question": similar_question.page_content,
            "query":examples_train[similar_question.page_content]
            }
            )
        
    result = generator.generate_test_query(question=question, tables=schema, examples=examples, foreign_keys=foreign_keys)
    break
    res.append(result)

  0%|          | 0/282 [00:01<?, ?it/s]


In [76]:
test[1865]

{'db_id': 'planet_1',
 'query': 'SELECT T1.Level FROM Has_Clearance AS T1 JOIN Employee AS T2 ON T1.Employee = T2.EmployeeID WHERE T2.position  =  "Physician";',
 'query_toks': ['SELECT',
  'T1.Level',
  'FROM',
  'Has_Clearance',
  'AS',
  'T1',
  'JOIN',
  'Employee',
  'AS',
  'T2',
  'ON',
  'T1.Employee',
  '=',
  'T2.EmployeeID',
  'WHERE',
  'T2.position',
  '=',
  '``',
  'Physician',
  "''",
  ';'],
 'query_toks_no_value': ['select',
  't1',
  '.',
  'level',
  'from',
  'has_clearance',
  'as',
  't1',
  'join',
  'employee',
  'as',
  't2',
  'on',
  't1',
  '.',
  'employee',
  '=',
  't2',
  '.',
  'employeeid',
  'where',
  't2',
  '.',
  'position',
  '=',
  'value'],
 'question': 'What level is Physician?',
 'question_toks': ['What', 'level', 'is', 'Physician', '?'],
 'sql': {'from': {'table_units': [['table_unit', 3], ['table_unit', 0]],
   'conds': [[False, 2, [0, [0, 13, False], None], [0, 1, False], None]]},
  'select': [False, [[0, [0, [0, 15, False], None]]]],
  '

In [74]:
' '.join(result.query.replace('\n',' ').replace(';',' ').split())

''

In [67]:
r1 = []
for r in res:
    r1.append(' '.join(r.query.replace('\n',' ').replace(';',' ').split()))

In [68]:
with open('./test.txt','w') as f:
    f.write('\n'.join(r1))